<a href="https://colab.research.google.com/github/RebeccaKessler/Machine_Learning/blob/main/Codes/Combined_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install required packages
!pip install sentencepiece
!pip install accelerate -U
!pip install pandas numpy matplotlib
!pip install scikit-learn seaborn
!pip install transformers torch pandas scikit-learn
!pip install sacremoses

In [ ]:
#import required packages
from transformers import Trainer, TrainingArguments, CamembertTokenizer, CamembertForSequenceClassification, CamembertConfig, FlaubertTokenizer, FlaubertForSequenceClassification
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import pipeline
import joblib
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import KFold

In [ ]:
# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = str(self.data.iloc[idx]['sentence'])
        label = int(self.data.iloc[idx]['encoded_labels'])

        encoding = self.tokenizer.encode_plus(
            sentence,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# Load the dataset
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/training_data.csv'
data = pd.read_csv(url)

# Define label encoder
label_encoder = LabelEncoder()

# Fit label encoder and transform labels
data['encoded_labels'] = label_encoder.fit_transform(data['difficulty'])

# Split data into train and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Define the tokenizer
tokenizer = FlaubertTokenizer.from_pretrained('flaubert/flaubert_base_cased')

# K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
    print(f"Training fold {fold+1}")
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]

    train_dataset = CustomDataset(train_data, tokenizer)
    eval_dataset = CustomDataset(val_data, tokenizer)

    # Set up the Trainer
    model = FlaubertForSequenceClassification.from_pretrained('flaubert/flaubert_base_cased', num_labels=6)
    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold}',
        learning_rate=0.00005,
        num_train_epochs=7,
        per_device_train_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=f'./logs_fold_{fold}',
        logging_steps=10,
        evaluation_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=500,
        fp16=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics
    )

    # Train and evaluate the model
    trainer.train()
    eval_result = trainer.evaluate()
    accuracy_list.append(eval_result['eval_accuracy'])
    precision_list.append(eval_result['eval_precision'])
    recall_list.append(eval_result['eval_recall'])
    f1_list.append(eval_result['eval_f1'])

In [ ]:
# Compute overall statistics of the model
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
overall_precision = sum(precision_list) / len(precision_list)
overall_recall = sum(recall_list) / len(recall_list)
overall_f1 = sum(f1_list) / len(f1_list)

print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")
print(f"Overall F1 Score: {overall_f1:.4f}")

In [ ]:
# Load and preprocess the data
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/training_data.csv'
full_data = pd.read_csv(url)
label_encoder = LabelEncoder()
full_data['encoded_labels'] = label_encoder.fit_transform(full_data['difficulty'])

# Load CamemBERT model pre-trained
model = FlaubertForSequenceClassification.from_pretrained('flaubert/flaubert_base_cased', num_labels=6)

# Load the tokenizer
tokenizer = FlaubertTokenizer.from_pretrained('flaubert/flaubert_base_cased')

# Create a new dataset object with the entire data
final_dataset = CustomDataset(combined_data, tokenizer)

# Modify training arguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=0.00005,
    num_train_epochs=7,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="no",
    save_strategy="steps",
    save_steps=500,
    fp16=True,
    )

# Re-initialize and train the Trainer with the new combined dataset
final_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset,
    compute_metrics=None
)

# Retrain the model on the whole dataset
final_trainer.train()

# Save the final trained model and tokenizer
model.save_pretrained('./flaubert_final_model')
tokenizer.save_pretrained('./flaubert_final_model')

In [ ]:
# Load and preprocess the data
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/training_data.csv'
data = pd.read_csv(url)
label_encoder = LabelEncoder()
data['encoded_labels'] = label_encoder.fit_transform(data['difficulty'])

In [ ]:
# Define the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
    print(f"Training fold {fold+1}")
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]

    train_dataset = CustomDataset(train_data, tokenizer)
    eval_dataset = CustomDataset(val_data, tokenizer)

    # Set up the Trainer
    model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)
    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold}',
        learning_rate=0.00015,
        num_train_epochs=7,
        per_device_train_batch_size=16,
        warmup_steps=1000,
        weight_decay=0.01,
        logging_dir=f'./logs_fold_{fold}',
        logging_steps=10,
        evaluation_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=500,
        fp16=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics
    )

    # Train and evaluate the model
    trainer.train()
    eval_result = trainer.evaluate()
    accuracy_list.append(eval_result['eval_accuracy'])
    precision_list.append(eval_result['eval_precision'])
    recall_list.append(eval_result['eval_recall'])
    f1_list.append(eval_result['eval_f1'])

In [ ]:
# Compute overall statistics of the model
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
overall_precision = sum(precision_list) / len(precision_list)
overall_recall = sum(recall_list) / len(recall_list)
overall_f1 = sum(f1_list) / len(f1_list)

print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")
print(f"Overall F1 Score: {overall_f1:.4f}")

In [ ]:
# Load and preprocess the data
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/training_data.csv'
full_data = pd.read_csv(url)
label_encoder = LabelEncoder()
full_data['encoded_labels'] = label_encoder.fit_transform(full_data['difficulty'])

#Load CamemBERT model pre-trained
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Create a new dataset object with the entire data
final_dataset = CustomDataset(full_data, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=0.00015,
    num_train_epochs=7,
    per_device_train_batch_size=16,
    warmup_steps=1000,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="no",
    save_strategy="steps",
    save_steps=500,
    fp16=True,
    )

# Re-initialize and train the Trainer with the new combined dataset
final_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset,
    compute_metrics=None
)

# Retrain the model on the whole dataset
final_trainer.train()

# Save the final trained model and tokenizer
model.save_pretrained('./camembert_final_model')
tokenizer.save_pretrained('./camembert_final_model')

In [ ]:
# Load the unlabelled data
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/unlabelled_test_data.csv'
unlabelled_data = pd.read_csv(url)

# Load the saved CamemBERT model and tokenizer
camembert_model_path = './camembert_final_model'
camembert_model = CamembertForSequenceClassification.from_pretrained(camembert_model_path)
camembert_tokenizer = CamembertTokenizer.from_pretrained(camembert_model_path)

# Load the saved Flaubert model and tokenizer
flaubert_model_path = './flaubert_final_model'
flaubert_model = FlaubertForSequenceClassification.from_pretrained(flaubert_model_path)
flaubert_tokenizer = FlaubertTokenizer.from_pretrained(flaubert_model_path)

# Create prediction pipelines to get probabilities
device = 0 if torch.cuda.is_available() else -1
camembert_classifier = pipeline('text-classification', model=camembert_model, tokenizer=camembert_tokenizer, framework='pt', device=device, return_all_scores=True)
flaubert_classifier = pipeline('text-classification', model=flaubert_model, tokenizer=flaubert_tokenizer, framework='pt', device=device, return_all_scores=True)

# Predict probabilities for the unlabelled data using both models
camembert_probs = camembert_classifier(unlabelled_data['sentence'].tolist())
flaubert_probs = flaubert_classifier(unlabelled_data['sentence'].tolist())

# Convert the predictions to numpy arrays
camembert_probs_array = np.array([[prob['score'] for prob in probs] for probs in camembert_probs])
flaubert_probs_array = np.array([[prob['score'] for prob in probs] for probs in flaubert_probs])

# Combine predictions using soft voting (average probabilities)
average_probs = (camembert_probs_array + flaubert_probs_array) / 2
final_predictions = np.argmax(average_probs, axis=1)

# Decode the numeric labels to original labels using the loaded LabelEncoder
predicted_labels = label_encoder.inverse_transform(final_predictions)

# Create a DataFrame to export
results_df = pd.DataFrame({
    'id': unlabelled_data['id'],
    'difficulty': predicted_labels
})

# Save the results to a new CSV file
results_df.to_csv('predicted_difficulties.csv', index=False)

print("Predictions saved to 'predicted_difficulties.csv'")
